<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/3_Sample_for_Model_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import ast

In [3]:
df_full_ra_rap = pd.read_excel("full_ra_rap.xlsx")

In [4]:
df_full_ra_rap

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
0,5821/18,2,"ע""פ 5821/18",2018,"ע""פ 5821/18","ע""פ",ערעור פלילי,0.0,2018-11-22,108,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2018
1,4217/18,10,"ע""פ 4217/18",2018,"ע""פ 4217/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
2,4344/18,2,"ע""פ 4344/18",2018,"ע""פ 4344/18","ע""פ",ערעור פלילי,0.0,2019-08-18,439,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2018
3,1993/18,4,"ע""פ 1993/18",2018,"ע""פ 1993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,1.0,"[['ע""פ 1993/18', 'ע""פ 2017/18']]",NaN,NaN,NaN,NaN,2018
4,4993/18,4,"ע""פ 4993/18",2018,"ע""פ 4993/18","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20605,7505/21,1,"רע""פ 7505/21 טל גויחמו נ. מדינת ישראל",2021,"רע""פ 7505/21","רע""פ",רשות ערעור פלילי,0.0,2021-11-11,4,...,"['שופט', 'שופט']","['תעבורה', 'מחוזי ']","['פתח תקווה', 'מרכז-לוד']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20606,513/21,3,"רע""פ 513/21 עידנ יהושע נ. מדינת ישראל",2021,"רע""פ 513/21","רע""פ",רשות ערעור פלילי,0.0,2021-03-22,56,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['באר שבע', 'אשקלון']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20607,225/21,1,"רע""פ 225/21 אלקאר דביר בע""מ נ. מדינת ישראל",2021,"רע""פ 225/21","רע""פ",רשות ערעור פלילי,0.0,2021-02-11,30,...,"['שופט', 'שופט']","['מחוזי ', 'עניינים מקומיים']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20608,4075/21,1,"רע""פ 4075/21 דב הרמנ נ. מדינת ישראל",2021,"רע""פ 4075/21","רע""פ",רשות ערעור פלילי,0.0,2021-06-10,0,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021


# Clean Doc's body - נקיון גוף המסמך

In [5]:
print((df_full_ra_rap["גוף המסמך"].iloc[2]))

['פסק דין בתיק ע"פ 4344/18 בבית המשפט העליון בשבתו כבית משפט לערעורים פליליים ע"פ 4344/18 לפני: כבוד השופט נ\' הנדל כבוד השופט ע\' גרוסקופף כבוד השופט א\' שטיין המערער: אשרף גאבר נגד המשיבה: מדינת ישראל ערעור על פסק דינו של בית המשפט המחוזי מרכז-לוד מיום 17.4.2018 בת"פ 58957-05-17 (השופטת ר\' לורך, ס"נ) תאריך הישיבה: ט"ו בתמוז התשע"ט (18.07.2019) בשם המערער: עו"ד ניר שנידרמן בשם המשיבה: עו"ד עדי שגב פסק דין השופט א\' שטיין: 1. הערעור שלפנינו מופנה אל גזר הדין של בית המשפט המחוזי מרכז-לוד בת"פ 58957-05-17 (השופטת ר\' לורך, ס"נ, כתוארה אז) מיום 17.4.2018, אשר השית על המערער – בעקבות הרשעתו במסגרת הסדר טיעון בגרימת חבלה בכוונה מחמירה, עבירה לפי סעיף 329(א)(1) לחוק העונשין, התשל"ז-1977 (להלן: חוק העונשין), ובנשיאת נשק ותחמושת שלא כדין, עבירה לפי סעיף 144(ב) לחוק העונשין – עונשים כדלקמן: 5 שנות מאסר בפועל החל מיום מעצרו, 28.5.2017; 10 חודשי מאסר על תנאי לבל יעבור עבירת אלימות או עבירת נשק מסוג פשע בתוך 3 שנים מיום שחרורו מהמאסר; 6 חודשי מאסר על תנאי לבל יעבור עבירת אלימות או עבירת נשק מסוג 

In [6]:
# פונקציה להמרת טקסט שמופיע כרשימה למחרוזת רגילה
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            # מנסה להמיר את התוכן בתוך הסוגריים לרשימה אמיתית
            text_list = ast.literal_eval(text)
            # איחוד המחרוזות לרצף טקסט אחד
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

# פונקציה לניקוי התווים המיותרים
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)  # הפיכת רשימה למחרוזת
    elif isinstance(text, str):
        # הסרת תווי רווח מיותרים ותווים מיוחדים
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

# הדפסת שורות לא קריאות לפני המרה
print("שורות לא קריאות לפני המרה:")
print(df_full_ra_rap["גוף המסמך"].head())

# המרה של הטקסטים הלא קריאים לטקסטים קריאים ושמירה על הדאטה המקורי
df_full_ra_rap["גוף המסמך"] = df_full_ra_rap["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(df_full_ra_rap["גוף המסמך"].head())

# file_path = 'full_final_df_cleaned.csv'
# df_full_ra_rap.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


שורות לא קריאות לפני המרה:
0    ['פסק דין בתיק ע"פ 5821/18 בבית המשפט העליון ב...
1    ['פסק דין בתיק ע"פ 4217/18 בבית המשפט העליון ב...
2    ['פסק דין בתיק ע"פ 4344/18 בבית המשפט העליון ב...
3    ['פסק דין בתיק ע"פ 1993/18 בבית המשפט העליון ב...
4    ['פסק דין בתיק ע"פ 4993/18 בבית המשפט העליון ב...
Name: גוף המסמך, dtype: object

שורות לאחר המרה:
0    ['פסק דין בתיק ע"פ 5821/18 בבית המשפט העליון ב...
1    ['פסק דין בתיק ע"פ 4217/18 בבית המשפט העליון ב...
2    ['פסק דין בתיק ע"פ 4344/18 בבית המשפט העליון ב...
3    ['פסק דין בתיק ע"פ 1993/18 בבית המשפט העליון ב...
4    ['פסק דין בתיק ע"פ 4993/18 בבית המשפט העליון ב...
Name: גוף המסמך, dtype: object


# DCA Files only

In [7]:
df_full_rap = df_full_ra_rap[df_full_ra_rap["סוג הליך"]=='רע"פ']
df_full_rap

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
494,5144/18,1,"רע""פ 5144/18",2018,"רע""פ 5144/18","רע""פ",רשות ערעור פלילי,0.0,2018-07-19,14,...,"['שופט', 'שופט']","['מחוזי ', 'עניינים מקומיים']","['תל אביב', 'בת ים']",NaN,NaN,NaN,NaN,NaN,NaN,2018
495,1746/18,3,"רע""פ 1746/18 אלחננ יהודה פלהיימר נ. מדינת ישראל",2018,"רע""פ 1746/18","רע""פ",רשות ערעור פלילי,0.0,2018-04-29,57,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['מרכז-לוד', 'פתח-תקווה']",NaN,NaN,NaN,NaN,NaN,NaN,2018
496,4262/18,4,"רע""פ 4262/18 רעננ בנ-טובימ נ. מדינת ישראל- עיר...",2018,"רע""פ 4262/18","רע""פ",רשות ערעור פלילי,0.0,2018-08-08,68,...,['שופט'],['מחוזי '],['תל אביב'],NaN,NaN,NaN,NaN,NaN,NaN,2018
497,1654/18,3,"רע""פ 1654/18 לאה מזרחי נ. מדינת ישראל",2018,"רע""פ 1654/18","רע""פ",רשות ערעור פלילי,0.0,2018-07-29,149,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2018
498,5548/18,4,"רע""פ 5548/18",2018,"רע""פ 5548/18","רע""פ",רשות ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20605,7505/21,1,"רע""פ 7505/21 טל גויחמו נ. מדינת ישראל",2021,"רע""פ 7505/21","רע""פ",רשות ערעור פלילי,0.0,2021-11-11,4,...,"['שופט', 'שופט']","['תעבורה', 'מחוזי ']","['פתח תקווה', 'מרכז-לוד']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20606,513/21,3,"רע""פ 513/21 עידנ יהושע נ. מדינת ישראל",2021,"רע""פ 513/21","רע""פ",רשות ערעור פלילי,0.0,2021-03-22,56,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['באר שבע', 'אשקלון']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20607,225/21,1,"רע""פ 225/21 אלקאר דביר בע""מ נ. מדינת ישראל",2021,"רע""פ 225/21","רע""פ",רשות ערעור פלילי,0.0,2021-02-11,30,...,"['שופט', 'שופט']","['מחוזי ', 'עניינים מקומיים']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021
20608,4075/21,1,"רע""פ 4075/21 דב הרמנ נ. מדינת ישראל",2021,"רע""פ 4075/21","רע""פ",רשות ערעור פלילי,0.0,2021-06-10,0,...,"['הרכב', 'שופט']","['מחוזי ', 'שלום']","['תל אביב', 'תל אביב']",NaN,NaN,NaN,NaN,NaN,NaN,2021


In [8]:
years =[2020,2015,2010,2005,2000]

In [9]:
sample_df = df_full_ra_rap[df_full_ra_rap["שנת פתיחת ההליך"].isin(years)]
sample_df

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
2430,2888/15,3,"ע""פ 2888/15",2015,"ע""פ 2888/15","ע""פ",ערעור פלילי,0.0,2017-01-02,610,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2431,1101/15,5,"ע""פ 1101/15",2015,"ע""פ 1101/15","ע""פ",ערעור פלילי,0.0,2015-10-02,216,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2432,2805/15,3,"ע""פ 2805/15",2015,"ע""פ 2805/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
2433,1961/15,2,"ע""פ 1961/15",2015,"ע""פ 1961/15","ע""פ",ערעור פלילי,0.0,2016-06-07,447,...,['הרכב'],['מחוזי '],['באר שבע'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2434,6963/15,3,"ע""פ 6963/15",2015,"ע""פ 6963/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18398,6795/00,3,"רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ",2000,"רע""פ 6795/00","רע""פ",רשות ערעור פלילי,0.0,2001-07-05,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18399,1492/00,3,"רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק",2000,"רע""פ 1492/00","רע""פ",רשות ערעור פלילי,0.0,2000-07-23,146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18400,3925/00,1,"רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי",2000,"רע""פ 3925/00","רע""פ",רשות ערעור פלילי,0.0,2000-09-11,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18417,29/01,4,"ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",2000,"ע""פ 29/01","ע""פ",ערעור פלילי,0.0,2002-12-02,701,...,['הרכב'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2001


In [10]:
sample_df.shape[0]

4346

# Drop null

In [11]:
sample_df[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
sample_df[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
sample_df[["מספר הליך","שם הליך"]].drop_duplicates(inplace=True, ignore_index=True)
sample_df

<ipython-input-11-9f43abb7fdde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
<ipython-input-11-9f43abb7fdde>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
<ipython-input-11-9f43abb7fdde>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df[["מספר הליך","שם הליך"]].d

,מספר תיק,מספר החלטות בתיק,שם הליך,שנת פתיחת ההליך,מספר הליך,סוג הליך,פירוט סוג הליך,תיק חסוי,תאריך אירוע אחרון,משך התיק בימים,...,סוג גורם שיפוטי דלמטה,סוג בית משפט דלמטה,מקום בית משפט דלמטה,הליך מאוחד,שמות ההליכים המאוחדים,צדדים נוספים,מספר תיק מלא,מספר השופטים,ניתנה החלטה כמבוקש הבקשה,Year
2430,2888/15,3,"ע""פ 2888/15",2015,"ע""פ 2888/15","ע""פ",ערעור פלילי,0.0,2017-01-02,610,...,['שופט'],['מחוזי '],['מרכז-לוד'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2431,1101/15,5,"ע""פ 1101/15",2015,"ע""פ 1101/15","ע""פ",ערעור פלילי,0.0,2015-10-02,216,...,['שופט'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2432,2805/15,3,"ע""פ 2805/15",2015,"ע""פ 2805/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
2433,1961/15,2,"ע""פ 1961/15",2015,"ע""פ 1961/15","ע""פ",ערעור פלילי,0.0,2016-06-07,447,...,['הרכב'],['מחוזי '],['באר שבע'],NaN,NaN,NaN,NaN,NaN,NaN,2015
2434,6963/15,3,"ע""פ 6963/15",2015,"ע""פ 6963/15","ע""פ",ערעור פלילי,1.0,1900-01-01,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18398,6795/00,3,"רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ",2000,"רע""פ 6795/00","רע""פ",רשות ערעור פלילי,0.0,2001-07-05,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18399,1492/00,3,"רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק",2000,"רע""פ 1492/00","רע""פ",רשות ערעור פלילי,0.0,2000-07-23,146,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18400,3925/00,1,"רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי",2000,"רע""פ 3925/00","רע""פ",רשות ערעור פלילי,0.0,2000-09-11,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000
18417,29/01,4,"ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",2000,"ע""פ 29/01","ע""פ",ערעור פלילי,0.0,2002-12-02,701,...,['הרכב'],['מחוזי '],['חיפה'],NaN,NaN,NaN,NaN,NaN,NaN,2001


In [12]:
cut_sample_df = sample_df[["גוף המסמך","מספר הליך","שם הליך"]].reset_index()
cut_sample_df

,index,גוף המסמך,מספר הליך,שם הליך
0,2430,"['החלטה בתיק ע""פ 2888/15 בבית המשפט העליון בשב...","ע""פ 2888/15","ע""פ 2888/15"
1,2431,"['פסק דין בתיק ע""פ 1101/15 בבית המשפט העליון ב...","ע""פ 1101/15","ע""פ 1101/15"
2,2432,"['פסק דין בתיק ע""פ 2805/15 בבית המשפט העליון ב...","ע""פ 2805/15","ע""פ 2805/15"
3,2433,"['פסק דין בתיק ע""פ 1961/15 בבית המשפט העליון ב...","ע""פ 1961/15","ע""פ 1961/15"
4,2434,"['פסק דין בתיק ע""פ 6963/15 בבית המשפט העליון ב...","ע""פ 6963/15","ע""פ 6963/15"
...,...,...,...,...
4341,18398,"[' בבית המשפט העליון בג""ץ6795/00 - ב\' בפני: כ...","רע""פ 6795/00","רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ"
4342,18399,"[' בבית המשפט העליון בה""נ 1492/00 בפני כבוד נש...","רע""פ 1492/00","רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק"
4343,18400,"[' בבית המשפט העליון בה""נ 3925/00 בפני כבוד נש...","רע""פ 3925/00","רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי"
4344,18417,"['פסק דין בתיק ע""פ 29/01 בבית המשפט העליון בשב...","ע""פ 29/01","ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל"


In [13]:
cut_sample_df.to_csv("cut_sample_df.csv", index=False, encoding='utf-8')

# RUN FROM HERE!!!

Change runtime from here - T4 or A100 NO NEED HIGH RAM


# Cut missing ending files

## FIX THIS!!!


In [14]:
# הגדרת פונקציה לעיצוב טקסט
def format_text(text, line_length=80):
    """
    פורמט טקסט כך שיהיה נוח לקריאה עם שורות שאורכן מוגבל.
    """
    import textwrap
    return "\n".join(textwrap.wrap(text, width=line_length))


In [15]:
# ביטויים לבדיקה
keywords = ["העותק כפוף לשינויי עריכה וניסוח", "העתק מתאים",'ניתנההיום','ניתןהיום','ניתן היום','ניתנה היום', "העתקמתאים", "מזכיר ראשי","supreme.court.gov.il","מרכז מידע","מרכזמידע"]

# בדיקה אילו שורות מכילות אחד מהביטויים
rows_with_keyword = cut_sample_df["גוף המסמך"].apply(
    lambda x: any(keyword in x for keyword in keywords) if isinstance(x, str) else False
)

# סינון שורות שלא מכילות את הביטוי
filtered_data = cut_sample_df[rows_with_keyword]

# הדפסת השורות שלא מכילות את הביטוי
rows_without_keyword = cut_sample_df[~rows_with_keyword]
# print("שורות שלא מכילות את הביטוי:\n")

# for index, row in rows_without_keyword.iterrows():
#     text = row.get("גוף המסמך", "")
#     if isinstance(text, str) and text:  # בדיקה אם הטקסט קיים ואינו ריק
#         print(f"אינדקס: {index}")
#         formatted_text = "\n".join(text[i:i+80] for i in range(0, len(text), 80))
#         print(formatted_text)
#         print("------------------------------------------------------------")
#     else:
#         print(f"אינדקס: {index} - שדה 'גוף המסמך' חסר או ריק.")
#         print("------------------------------------------------------------")

# הדפסת מספר השורות שנמחקו
num_removed = len(rows_without_keyword)
print(f"\nמספר השורות שנמחקו: {num_removed}")

# שמירת הנתונים המסוננים לקובץ חדש
filtered_data.to_csv("new_filtered_data.csv", index=False, encoding='utf-8')
print("\nהנתונים המסוננים נשמרו לקובץ: 'new_filtered_data.csv'")

# עדכון הטבלה המקורית
cut_sample_df = filtered_data



מספר השורות שנמחקו: 294

הנתונים המסוננים נשמרו לקובץ: 'new_filtered_data.csv'


In [16]:
# פונקציה לחיתוך הטקסט בהתאם לכללים
def trim_text(text):
    min_length = 410
    last_sen = 1

    if len(text) > min_length:
        return text[-min_length:-last_sen]  # חיתוך החל מ-min_length תווים מהסוף ועד last_sen תווים מהסוף

    return text[:-last_sen]  # חיתוך עד last_sen תווים מהסוף אם הטקסט קצר מ-min_length

# הסרת שורות עם ערכים חסרים בעמודת "גוף המסמך"
cut_sample_df = cut_sample_df.dropna(subset=["גוף המסמך"]).copy()

# חיתוך הטקסטים בעמודת "גוף המסמך"
cut_sample_df["גוף המסמך חתוך"] = cut_sample_df["גוף המסמך"].apply(trim_text)

# שמירת האינדקס המקורי
cut_sample_df.reset_index(inplace=True, drop=False)


In [17]:
cut_sample_df.head()


,level_0,index,גוף המסמך,מספר הליך,שם הליך,גוף המסמך חתוך
0,0,2430,"['החלטה בתיק ע""פ 2888/15 בבית המשפט העליון בשב...","ע""פ 2888/15","ע""פ 2888/15",כלכליות שמעשיו המיטו על משפחת הקורבן. 18. סוף ...
1,1,2431,"['פסק דין בתיק ע""פ 1101/15 בבית המשפט העליון ב...","ע""פ 1101/15","ע""פ 1101/15","בר, הערעור על הכרעת הדין נדחה. הערעור על גזר ה..."
2,2,2432,"['פסק דין בתיק ע""פ 2805/15 בבית המשפט העליון ב...","ע""פ 2805/15","ע""פ 2805/15","לדחות את הערעור, ולהותיר את גזר הדין של בית המ..."
3,3,2433,"['פסק דין בתיק ע""פ 1961/15 בבית המשפט העליון ב...","ע""פ 1961/15","ע""פ 1961/15",ירתם המערער לטיפול מתאים בין כותלי הכלא ויתמיד...
4,4,2434,"['פסק דין בתיק ע""פ 6963/15 בבית המשפט העליון ב...","ע""פ 6963/15","ע""פ 6963/15","בחון ומיון של שירות בתי הסוהר, בטלפונים: 08-97..."


In [18]:
cut_sample_df.to_csv("cut_sample_df.csv", index=False, encoding='utf-8')


CHANGE RUNTIME HERE

In [19]:
cut_sample_df = pd.read_csv("cut_sample_df.csv")
cut_sample_df


,level_0,index,גוף המסמך,מספר הליך,שם הליך,גוף המסמך חתוך
0,0,2430,"['החלטה בתיק ע""פ 2888/15 בבית המשפט העליון בשב...","ע""פ 2888/15","ע""פ 2888/15",כלכליות שמעשיו המיטו על משפחת הקורבן. 18. סוף ...
1,1,2431,"['פסק דין בתיק ע""פ 1101/15 בבית המשפט העליון ב...","ע""פ 1101/15","ע""פ 1101/15","בר, הערעור על הכרעת הדין נדחה. הערעור על גזר ה..."
2,2,2432,"['פסק דין בתיק ע""פ 2805/15 בבית המשפט העליון ב...","ע""פ 2805/15","ע""פ 2805/15","לדחות את הערעור, ולהותיר את גזר הדין של בית המ..."
3,3,2433,"['פסק דין בתיק ע""פ 1961/15 בבית המשפט העליון ב...","ע""פ 1961/15","ע""פ 1961/15",ירתם המערער לטיפול מתאים בין כותלי הכלא ויתמיד...
4,4,2434,"['פסק דין בתיק ע""פ 6963/15 בבית המשפט העליון ב...","ע""פ 6963/15","ע""פ 6963/15","בחון ומיון של שירות בתי הסוהר, בטלפונים: 08-97..."
...,...,...,...,...,...,...
4047,4341,18398,"[' בבית המשפט העליון בג""ץ6795/00 - ב\' בפני: כ...","רע""פ 6795/00","רע""פ 6795/00 שינוי התנועה החילונית נ. שר הפנימ","בג""ץ6795/00 - ב\' בפני: כבוד השופט א\' מצא העו..."
4048,4342,18399,"[' בבית המשפט העליון בה""נ 1492/00 בפני כבוד נש...","רע""פ 1492/00","רע""פ 1492/00 ילנה גורליק נ. אלכסנדר גורליק","עץ המשפטי לממשלה: עו""ד זיוה שמחה החלטה בדבר קב..."
4049,4343,18400,"[' בבית המשפט העליון בה""נ 3925/00 בפני כבוד נש...","רע""פ 3925/00","רע""פ 3925/00 זרצקי אירינה נ. זרצקי אנדרי","המבקשת: עו""ד איליה וייסברג בשםהיועץ המשפטי לממ..."
4050,4344,18417,"['פסק דין בתיק ע""פ 29/01 בבית המשפט העליון בשב...","ע""פ 29/01","ע""פ 29/01 אמינ סעיד חילו נ. מדינת ישראל",טת ד\' ביניש: אני מסכימה. ש ו פ ט ת השופטת א\'...


# Embedding


# Model prediction

In [ ]:
from transformers import AutoTokenizer, AutoModel


In [20]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import joblib
import pandas as pd

# טעינת המודל והטוקנייזר
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

# פונקציה שמקבלת טקסט ומחזירה את האימבדינג כוקטור
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

# בדיקה אם עמודת "גוף המסמך חתוך" קיימת ומכילה ערכים תקינים
if "גוף המסמך חתוך" not in cut_sample_df.columns or cut_sample_df["גוף המסמך חתוך"].isna().all():
    raise ValueError("עמודת 'גוף המסמך חתוך' חסרה או ריקה. יש לוודא חיתוך טקסטים תקין לפני המשך.")

# חישוב האימבדינגים עבור כל שורה בעמודת "גוף המסמך חתוך"
print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך...")
embeddings_new = np.array([get_embeddings(text) for text in cut_sample_df["גוף המסמך חתוך"]])
print("אימבדינגים חושבו בהצלחה!")

# טוען את המודל והמשאבים
classifier = joblib.load("classifier_model.pkl")
optimal_threshold = joblib.load("optimal_threshold.pkl")

print("המודל והמשאבים נטענו בהצלחה.")

# חיזוי הסתברויות
print("מתחילים לחזות הסתברויות...")
predicted_probabilities = classifier.predict_proba(embeddings_new)[:, 1]
print("הסתברויות נחזו בהצלחה!")

# חישוב Threshold מותאם ליחס הכיתות המקורי (94% Class 0 ו-6% Class 1)
sorted_probabilities = np.sort(predicted_probabilities)
threshold_for_original_ratio = sorted_probabilities[int(len(sorted_probabilities) * 0.94)]
print(f"Threshold מותאם ליחס המקורי (94%-6%): {threshold_for_original_ratio}")

# הוספת עמודת ההסתברויות
cut_sample_df["predicted_probability"] = predicted_probabilities

# יצירת תחזיות לפי threshold אופטימלי
cut_sample_df["prediction_optimal_threshold"] = (predicted_probabilities >= optimal_threshold).astype(int)

# יצירת תחזיות לפי יחס הכיתות המקורי
cut_sample_df["prediction_original_ratio"] = (predicted_probabilities >= threshold_for_original_ratio).astype(int)

# בדיקת התפלגות התחזיות לפי threshold המקורי
original_ratio_distribution = cut_sample_df["prediction_original_ratio"].value_counts(normalize=True)
print("התפלגות תחזיות לפי יחס המחלקות המקורי:")
print(original_ratio_distribution)

# שמירה של התחזיות והנתונים לקובץ Excel
output_file = "full_data_with_predictions.xlsx"
cut_sample_df.to_excel(output_file, index=False, engine="openpyxl")
print(f"הטבלה המלאה עם הפרדיקציות נשמרה בקובץ: '{output_file}'")
from transformers import AutoTokenizer, AutoModel
import numpy as np
import joblib
import pandas as pd

# טעינת המודל והטוקנייזר
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

# פונקציה שמקבלת טקסט ומחזירה את האימבדינג כוקטור
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

# בדיקה אם עמודת "גוף המסמך חתוך" קיימת ומכילה ערכים תקינים
if "גוף המסמך חתוך" not in cut_sample_df.columns or cut_sample_df["גוף המסמך חתוך"].isna().all():
    raise ValueError("עמודת 'גוף המסמך חתוך' חסרה או ריקה. יש לוודא חיתוך טקסטים תקין לפני המשך.")

# חישוב האימבדינגים עבור כל שורה בעמודת "גוף המסמך חתוך"
print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך...")
embeddings_new = np.array([get_embeddings(text) for text in cut_sample_df["גוף המסמך חתוך"]])
print("אימבדינגים חושבו בהצלחה!")

# טוען את המודל והמשאבים
classifier = joblib.load("classifier_model.pkl")
optimal_threshold = joblib.load("optimal_threshold.pkl")

print("המודל והמשאבים נטענו בהצלחה.")

# חיזוי הסתברויות
print("מתחילים לחזות הסתברויות...")
predicted_probabilities = classifier.predict_proba(embeddings_new)[:, 1]
print("הסתברויות נחזו בהצלחה!")

# חישוב Threshold מותאם ליחס הכיתות המקורי (94% Class 0 ו-6% Class 1)
sorted_probabilities = np.sort(predicted_probabilities)
threshold_for_original_ratio = sorted_probabilities[int(len(sorted_probabilities) * 0.94)]
print(f"Threshold מותאם ליחס המקורי (94%-6%): {threshold_for_original_ratio}")

# הוספת עמודת ההסתברויות
cut_sample_df["predicted_probability"] = predicted_probabilities

# יצירת תחזיות לפי threshold אופטימלי
cut_sample_df["prediction_optimal_threshold"] = (predicted_probabilities >= optimal_threshold).astype(int)

# יצירת תחזיות לפי יחס הכיתות המקורי
cut_sample_df["prediction_original_ratio"] = (predicted_probabilities >= threshold_for_original_ratio).astype(int)

# בדיקת התפלגות התחזיות לפי threshold המקורי
original_ratio_distribution = cut_sample_df["prediction_original_ratio"].value_counts(normalize=True)
print("התפלגות תחזיות לפי יחס המחלקות המקורי:")
print(original_ratio_distribution)

# שמירה של התחזיות והנתונים לקובץ Excel
output_file = "full_data_with_predictions.xlsx"
cut_sample_df.to_excel(output_file, index=False, engine="openpyxl")
print(f"הטבלה המלאה עם הפרדיקציות נשמרה בקובץ: '{output_file}'")

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
module 'sympy.printing' has no attribute 'str'

In [ ]:
cut_sample_df

In [ ]:
# שמירה של התחזיות והנתונים לקובץ Excel
output_file = "full_data_with_predictions.xlsx"
cut_sample_df.to_excel(output_file, index=False, engine='openpyxl')
print(f"הטבלה המלאה עם הפרדיקציות נשמרה בקובץ: '{output_file}'")


In [ ]:
cut_sample_df

In [ ]:
cut_sample_df.shape


# Analysis

In [ ]:
print(f"Current Threshold: {optimal_threshold}")


In [ ]:
import matplotlib.pyplot as plt

# גרף התפלגות ההסתברויות
plt.hist(predicted_probabilities, bins=50, alpha=0.7)
plt.title("Distribution of Predicted Probabilities")
plt.xlabel("Probability")
plt.ylabel("Frequency")
plt.show()


In [ ]:
import numpy as np

# חישוב Threshold מותאם ליחס הכיתות המקורי (94% Class 0 ו-6% Class 1)
sorted_probabilities = np.sort(predicted_probabilities)
threshold_for_original_ratio = sorted_probabilities[int(len(sorted_probabilities) * 0.94)]

# הצגת ה-Threshold המחושב
print(f"Threshold for original ratio (94% Class 0, 6% Class 1): {threshold_for_original_ratio}")

# יצירת תחזיות לפי ה-Threshold המחושב
predictions_original_ratio = (predicted_probabilities >= threshold_for_original_ratio).astype(int)

# בדיקת התפלגות התחזיות
predictions_distribution = pd.Series(predictions_original_ratio).value_counts(normalize=True)
predictions_distribution *= 100  # להמיר לאחוזים

# הצגת התפלגות התחזיות
print("\nPredictions Distribution (%):")
print(predictions_distribution)

# אם תרצה להוסיף את התחזיות לטבלה (אופציונלי)
cut_sample_df["prediction_original_ratio"] = predictions_original_ratio


In [ ]:
binary_counts = cut_sample_df['prediction'].value_counts()
print(binary_counts)
labels = ['Denied (0)','Granted (1)']

plt.figure(figsize=(6, 6))
plt.pie(binary_counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=['#66b3ff', '#ff9999'])
plt.title('Outcome of Case (Binary Representation)')

plt.show()